In [48]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

# 많은 prompt들을 하나로 합치게 해줌
from langchain.prompts.pipeline import PipelinePromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 역할극 어시스턴트로서 캐릭터를 흉내내는 AI
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

# 어떻게 질문하고 답변해야 하는지 예시를 드는 프롬프트
example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

# AI 도우미가 우리의 텍스트를 완성해주는 프롬프트
start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

# 3개의 prompt를 하나로 합쳐주는 prompt
final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),  # 앞에 있는 Key는 반드시 prompt 이름과 일치해야 함
    ("example", example),
    ("start", start),
]

# 모든 템플릿을 하나로 합치기
full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

# full_prompt의 형식 확인
# full_prompt.format(
#     character="Pirate",
#     example_question="What is your location?",
#     example_answer="Arrrrg! That is a secret!! Arg arg!!",
#     question="What is your fav food?",
# )

chain = full_prompt | chat

# 실제 AI 활용
chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothing beats the taste o' the sea! What about ye, landlubber?

AIMessageChunk(content="Arrr matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, nothing beats the taste o' the sea! What about ye, landlubber?")